In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import numpy as np
import math
import pathlib
import pandas as pd
import os

from tensorflow import keras

import plotly.express as px
import plotly.graph_objects as go

import datetime


In [ ]:
# Continuar desenvolvimento:
# https://plotly.com/python/line-charts/#line-plots-with-column-encoding-color

def show_copare_prediction():
	fig = px.line(df, x='index', y='predictions', title='Ibova predito')
	fig.add_scatter(x=df['index'], y=df['real_values'], mode='lines')
	fig.show()

def show_distribution_error():
	fig = px.histogram(df,x='error', )
	fig.show()


def RMSE(Y_actual,Y_Predicted):

	rmse = mean_squared_error(Y_actual, Y_Predicted, squared = False)
	return rmse


def MAPE(actual, predicted) -> float:
	return round(np.mean(np.abs(( actual - predicted) / actual)) * 100, 2)



In [ ]:
# Reading DF teste
PATH_TESTE = '../Data/3_Gold/Teste_all_stocks.csv'
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE

test_dataset = test_dataset.drop(['oil_5','usd_5','abev_5','jbs_5','petr_5','vale_5',],axis=1)
test_labels = test_dataset.pop('ibova_5')

In [ ]:
TYPE_MODEL = 'CNN'
# MODELS_PATH = f'../Experiments/Models/{TYPE_MODEL}/'
MODELS_PATH = f'../Experiments/Models/{TYPE_MODEL}/'
SAVE_TIME = datetime.datetime.strftime(datetime.datetime.now(), "%y_%m_%d_%Hh%Mm%S")

models = os.listdir(MODELS_PATH)

# Carregando os modelos 
df_results = pd.DataFrame(columns=['path_model','MAPE','RMSE','model_type'])

for id, model_path in enumerate(models):
	print(model_path)

	model = keras.models.load_model(f'../Experiments/Models/{TYPE_MODEL}/{model_path}')
	# Model\Models\MLP
	test_predictions = model.predict(test_dataset).flatten()

	# Definindo o Dfs
	df_predic = pd.DataFrame(test_predictions, columns=['predictions'])
	df_predic = df_predic.reset_index()

	df_label = pd.DataFrame(test_labels.values, columns=['real_values'])
	df_label = df_label.reset_index()

	# Validação da quantidade de resultados
	# predict_len = len(test_predictions)
	# print(f'Predict len: {predict_len}')

	error = test_predictions - test_labels
	df_error = pd.DataFrame(error.values, columns=['error'])
	df_error = df_error.reset_index()

	# Merging Df
	df = df_predic.merge(df_label)
	df = df.merge(df_error)

	# Teste 
	# df.loc[id,'model_type'] = TYPE_MODEL
	# df.to_csv(f'Predictions_{TYPE_MODEL}_{SAVE_TIME}.csv',header=True,)
	# exit()

	# Get metrics and saving results
	mape = MAPE(df['real_values'], df['predictions'])
	rmse = RMSE(df['real_values'], df['predictions'])
	print(mape)
	print(rmse)

	df_results.loc[id,'path_model'] = model_path
	df_results.loc[id,'MAPE'] = mape
	df_results.loc[id,'RMSE'] = rmse
	df_results.loc[id,'model_type'] = TYPE_MODEL
	# df_results = df_results.reset_index()


In [ ]:
df_results.to_csv(f'Indicators_{TYPE_MODEL}_{SAVE_TIME}.csv',header=True,)
# df_results = None

In [ ]:
show_copare_prediction()

In [ ]:
show_distribution_error()